# Assigning Paths and Imports

In [12]:
# -----------------------------------------------------------------------------
# Copyright 2020 by ShabaniPy Authors, see AUTHORS for more details.
#
# Distributed under the terms of the MIT license.
#
# The full license is in the file LICENCE, distributed with this software.
# -----------------------------------------------------------------------------

"""Template for general aggregation and processing of data for analysis.

"""

import os

import numpy as np
import h5py
import matplotlib

from shabanipy.bulk.data_classifying import (
    DataClassifier,
    FilenamePattern,
    LogPattern,
    MeasurementPattern,
    NamePattern,
    RampPattern,
    StepPattern,
    ValuePattern,
)

from shabanipy.bulk.data_processing import (
    PreProcessingStep,
    ProcessCoordinator,
    ProcessingStep,
    SummarizingStep,
)

YEAR = '2022'
MONTH = '08'
MONTHDAY = '0822'

#: List of path to inspect looking for labber hdf5 data. Path is directed to google drive folder where labber data is stored
FOLDERS = ["/Volumes/GoogleDrive/.shortcut-targets-by-id/1p4A2foj_vBr4k6wrGEIpURdVkG-qAFgb/nyu-quantum-engineering-lab/"
           f"labber/data-backups/qubitfridge/Data/{YEAR}/{MONTH}/Data_{MONTHDAY}"]

#: Name of the sample that must appear in the measurement name usually of the form "{Wafer}-{Piece}_{Design}-{Iteration}_{Junction}_{Cooldown}"
SAMPLE_ID = "JS626-4SQ-1-001-CD3"

#: hdf5 file number 
FILE_NUM = '034'


#: Path to store generated files
PATH = (f"/Users/bh/Desktop/Code/Topological JJ/Samples/{SAMPLE_ID}")

#: Name of generated raw data file
RAW_DATA_NAME = (f"{PATH}/Data/{SAMPLE_ID}_aggregated-data-{FILE_NUM}.hdf5")

#: Name of generated processed data file
PROCESSED_DATA_NAME = (f"{PATH}/Data/{SAMPLE_ID}_processed-data-{FILE_NUM}.hdf5")

# Data Aggregation

In [27]:
# --- Configuation ---------------------------------------------------------------------

clf = DataClassifier(
    patterns=[
        MeasurementPattern(
            name="Data",
            filename_pattern=FilenamePattern(
                regex=f"{SAMPLE_ID}-{FILE_NUM}",
            ),
            patterns=[
                # StepPattern(
                #     name="Temperature",
                #     name_pattern=NamePattern(names=["Fridge - Temperature closed loop - target temperature"]),
                #     use_in_classification=True,
                #     classifier_level=3,
                # ),
                StepPattern(
                    name="External Attenuation",
                    name_pattern=NamePattern(names=["External Attenuation"]),
                    use_in_classification=True,
                    classifier_level=1,
                 ),
                StepPattern(
                    name="VNA Power",
                    name_pattern=NamePattern(names=["VNA - Output power"]),
                    use_in_classification=True,
                    classifier_level=1,
                 ),
                
                StepPattern(
                    name="RS2A",
                    name_pattern=NamePattern(names=["RS2A"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="RS2B",
                    name_pattern=NamePattern(names=["RS2B"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="RS3A",
                    name_pattern=NamePattern(names=["RS3A"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="RS3B",
                    name_pattern=NamePattern(names=["RS3B"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="RS4A",
                    name_pattern=NamePattern(names=["RS4A"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="RS4B",
                    name_pattern=NamePattern(names=["RS4B"]),
                    use_in_classification=True,
                    classifier_level=2,
                 ),
                StepPattern(
                    name="for-back",
                    name_pattern=NamePattern(names=["Foward-Backward Sweep"]),
                    use_in_classification=True,
                    classifier_level=3,
                 ),
#                 LogPattern(name="Voltage Drop", pattern=NamePattern(names=("DMM - SingleValue"))),
#                 LogPattern(name="dR", pattern=NamePattern(names=("SRS3 - Value")))
                LogPattern(name="S21", pattern=NamePattern(names=("VNA - S21")),x_name = "Frequency"),
                # LogPattern(name="power", x_name=("VNA - Output power")),
            ],
        ),

    ]
)

# --- Run ------------------------------------------------------------------------------

clf.identify_datasets(FOLDERS)

for n, datasets in clf._datasets.items():
    print(f"Measurement: {n}")
    for d in datasets:
        print(f"  - {d.rsplit(os.sep, 1)[-1]}")

clf.classify_datasets()
for n, ds in clf._classified_datasets.items():
    print(f"Measurement: {n}")
    for path, classifiers in ds.items():
        print(f"  - {d.rsplit(os.sep, 1)[-1]}")
        for level, values in classifiers.items():
            print(f"    - level {level}: {values}")

clf.consolidate_dataset(RAW_DATA_NAME)

# /Volumes/GoogleDrive/.shortcut-targets-by-id/1p4A2foj_vBr4k6wrGEIpURdVkG-qAFgb/nyu-quantum-engineering-lab/labber/data-backups/qubitfridge/Data/2022/07

Measurement: Data
  - JS626-4SQ-1-001-CD3-034.hdf5
Measurement: Data
  - JS626-4SQ-1-001-CD3-034.hdf5
    - level 1: {'External Attenuation': Classifier(column_name='External Attenuation', values=(0.0,), requires_filtering=False), 'VNA Power': Classifier(column_name='VNA - Output power', values=(-20.0,), requires_filtering=False)}
    - level 2: {'RS2A': Classifier(column_name='RS2A', values=(0.0,), requires_filtering=False), 'RS2B': Classifier(column_name='RS2B', values=(0.0,), requires_filtering=False), 'RS3A': Classifier(column_name='RS3A', values=(-6.0,), requires_filtering=False), 'RS3B': Classifier(column_name='RS3B', values=(-6.0,), requires_filtering=False), 'RS4A': Classifier(column_name='RS4A', values=(0.0,), requires_filtering=False), 'RS4B': Classifier(column_name='RS4B', values=(0.0,), requires_filtering=False)}
    - level 3: {'for-back': Classifier(column_name='Foward-Backward Sweep', values=(0.0, 1.0), requires_filtering=True)}
